In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("sync.csv", skipinitialspace=True)
df

In [ ]:
df["vec_size_bytes"] = df["vec_size"] * df["type_size"]

In [ ]:
df["runtime_normalized_seconds"] = df["runtime"] / (df["num_repetitions"] * 1e6)

In [ ]:
df["bytes_per_second"] = df["vec_size_bytes"] / df["runtime_normalized_seconds"]

In [ ]:
sns.set(rc={"figure.figsize":(12, 10)})
sns.lineplot(
    x="vec_size_bytes", 
    y="bytes_per_second", 
    hue="method_id", 
    #style="type_id"
    data=df[df["type_id"] == "sfixed32"]
)

In [ ]:
max(df_chunked["vec_size"].unique()

In [ ]:
df_chunked = df[df["chunk_size"] != 0]

In [ ]:
sns.set(rc={"figure.figsize":(12, 10)})
sns.lineplot(
    x="chunk_size", 
    y="bytes_per_second", 
    hue="method_id", 
    #style="type_id"
    data=df_chunked[(df_chunked["type_id"] == "sfixed32") & (df_chunked["vec_size"] == 1048576)]
)

In [ ]:
sns.set(rc={"figure.figsize":(12, 10)})
sns.lineplot(
    x="chunk_size", 
    y="bytes_per_second", 
    hue="method_id", 
    #style="type_id"
    data=df_chunked[(df_chunked["type_id"] == "float") & (df_chunked["vec_size"] == 1048576)]
)